# FT-IM Processor v0.95

### Elvin Cabrera and Brian H. Clowers
* Department of Chemistry
* Washington State University
* Pullman, WA 99164

### Release License: https://www.gnu.org/licenses/gpl-3.0.en.html

### This set of scripts accompanies the manuscript ???, 2021

### Reads mzML with the mobility information encoded in the time domain using linear frequency modulation

### Libraries needed for functionality:
`
panel 
matplotlib
numpy
os
glob
pandas
plotly
scipy
pyteomics
kaleido
`

### TODO:
* Add variable for bin size for obitrap loading
* Add expgaussian fit
* ~~Truncate large DC after transform~~
    * Complete as a user defined parameter...
* Integrate mobility information into peaks fits and dataframe
* Add checks for loading data and button functionality.
* Add reporting for when peak picking fails...
* Add AFT (Absorption Mode FT) and half windowing function
* ~~Add save plot image~~
* ~~Add csv save~~
* Debug area is overlapping with peak fit reporting...Add divider?

In [1]:
import panel as pn
# pn.extension()#sizing_mode = 'stretch_both')
pn.extension('plotly')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os, glob

import pandas as pd

import plotly.graph_objects as go

from scipy import signal
from scipy.signal import find_peaks
from scipy.interpolate import UnivariateSpline, interp1d
from pyteomics import ms1, mzml, auxiliary

import scipy.optimize as optimize
from scipy.optimize import leastsq
import collections

### Default Instrumental Values
* Use to speed processing

In [3]:
FREQSWEEP = 15 #Hz/s
DRIFTLENGTH = 100 #cm
DRIFTVOLTAGE = 1800 #Volts
TEMPERATURE = 298 #Temp (K)
PRESSURE = 3.5 #Pressure (torr)

DCCUTOFF = 50 #need to add to a slider later

PLOTOUTPUT = '.png' #alternate is '.svg' or '.pdf'

## Processing Code

Function to extract mass spectrum.

In [4]:
def getMS(spectra):
    '''
    Return the full mass spectrum. 
    
    Accepts the spectral class returned after reading the spectrum using pyteomics
    '''
    mzIndex,intens = [],[]
    mzIndex = spectra[0]['m/z array']#this assumes all the scans are the same in the m/z dimsions
    intens = np.zeros_like(spectra[0]['intensity array'])
    for i,s in enumerate(spectra):#Assumes keys are numbers\
        if s['ms level'] == 1:
            intens+=s['intensity array']

    mzIndex = np.array(mzIndex)
    intens = np.array(intens)
    
    return mzIndex, intens

In [5]:
def loadMS(fileName):
    '''
    Reads a mzML file for further processing.
    '''
    spectral_data = [s for s in mzml.read(fileName)]
    
    x, y = getMS(spectral_data)
    
    return x, y, spectral_data

In [6]:
def scanOrbi(fname, step=0.1, saveBool=False):
    """
    
    Adapted from scripts written by McCabe and Laganowsky (TAMU)
    
    Read and extract data from mzML coverted from Thermo *.RAW. (e.g. pyteomics)

    Over the acquision, mass spectrum and time from each scan from the *.RAW will
    be extracted. Each scan is referred to as a scan number. This data will be formatted
    in a dictionary, which is necessary for FFT. Importantly, the data will be regrided
    based on input paramaters, and this grid should be constant among datasets in particular
    for averaging.

    Parameters
    ----------
    fname : str
        Path to *.mzML data
    step : float
        Step size for m/z grid
    saveBool : bool
        Save pickle file of data object for processed *.RAW

    Returns
    -------
    dict
        A dict keyed by scan_number with values containing time and m/z intensity array. Note
        that the m/z grid, used to re-grid all m/z intensity arrays for each scan, is stored in
        dict only once, which can be accessed using key "mz_axis".   

    Do we need to add some check for the msLevel?

    """
    print("Processing %s" % fname)
    msruns = [s for s in mzml.read(fname)]
    scans = np.arange(0, len(msruns))#do we add 1 here
    mz_start = msruns[0]['scanList']['scan'][0]['scanWindowList']['scanWindow'][0]['scan window lower limit'].real
    mz_end = msruns[0]['scanList']['scan'][0]['scanWindowList']['scanWindow'][0]['scan window upper limit'].real
    
 
    # for storing data
    data = {}

    # set grid
    grid = np.arange(mz_start, mz_end, step)

    #for time, ms in zip(self.times, self.data):
    for i,s in enumerate(msruns):
        if i%100 == 0:
            print("Scan # %s"%i)
        # extract values
        ms = np.array(s['m/z array'])
        intensity = np.array(s['intensity array'])

#         #time = self.msrun.scan_time_from_scan_name(s)
        time = s['scanList']['scan'][0]['scan start time'].real*60

#         # regrid mz axis to grid
        ms = regrid(grid, np.column_stack((ms, intensity)), False)

#         # add to data
        data[i] = time, ms[::,1]

#     # store mz axis (or grid used)
    data["mz_axis"] = grid

    if saveBool:
        print("Not implemented")
        # now save the data
#         f = open(fname+'.pkl', 'wb')
#         pickle.dump(data,f)
#         f.close()

    return data

In [7]:
def regrid(grid, data, debugBool = False):
    """
    Re-grid data to grid using interpolation

    Parameters
    ----------
    grid : np_array
        grid to interpolate values
    data : np_array
        An array containing m/z values [(m/z1, int1), (m/z2, int2), ...]

    Returns
    -------
    np_array
        A numpy array containing values regrided using interpolation.  The array is
        formatted the same is data parameter i.e. [(m/z1, int1), (m/z2, int2), ...]
    """
    if debugBool:
        print(len(data), len(grid))#, data[:,0], data[:,1])
    
    f = interp1d(data[:,0], data[:,1], bounds_error=False, fill_value=0)
#     f = interp1d(data[0], data[-1], bounds_error=False, fill_value=0)
    inty = f(grid)
    return np.column_stack((grid, inty))

In [8]:
def loadOrbiMS_v2(fileName, scanStep = 0.1):
    '''
    Loads an orbitrap mzML file for FT-IM Processing.  
    As compared to a LTQ, the Orbitrap resolution is obviously much higher resolution which creates a processing problem for display.
    The data are stored in the mzML domain in sparse vector (i.e. non-evenly spaced points).  For display we need to fix that.
    To make this happen the high res data must be binned and depending on the resolution you choose the time for loading will scale.
    Smaller bins, longer load times.     
    
    '''
    
    #get raw data dictionary
    rawData = scanOrbi(fileName, scanStep)

    
    x = rawData['mz_axis']
    y = np.zeros_like(rawData[1][1])
    for j,k in enumerate(rawData.keys()):
        if k != 'mz_axis':
            y+=rawData[k][1]

    return x, y, rawData

Function to extract mobility spectrum.

In [9]:
def getXIC(spectra, mzVal, tol = 0.05, secondsBool = True):
    '''
    Get the extracted mobility spectrum.
    '''
    tIndex,intens = [],[]
    left,right = mzVal-tol, mzVal+tol
    for s in spectra:
        curTime = np.float(s['scanList']['scan'][0]['scan start time'])
        tIndex.append(curTime)
        
        curMZ = np.array(s['m/z array'], dtype = np.float)
        m = curMZ.searchsorted(left)
        n = curMZ.searchsorted(right)
        intens.append(s['intensity array'][m:n].sum())
    if secondsBool:
        return np.array(tIndex)*60.0, np.array(intens)
    else:
        return np.array(tIndex), np.array(intens)

In [10]:
def getXIC_Orbi(rawData, mzVal, tol = 0.05, secondsBool = True):
    '''
    Get the extracted mobility spectrum.
    
    There are some subtle differences between the dictionaries between the orbi and LTQ datasets. 
    '''
    tIndex,intens = [],[]
    left,right = mzVal-tol, mzVal+tol
    for i,k in enumerate(rawData.keys()):
        if k != 'mz_axis':
            curTime = rawData[k][0]
            tIndex.append(curTime)

            curMZ = rawData['mz_axis']
            m = curMZ.searchsorted(left)
            n = curMZ.searchsorted(right)
            intens.append(rawData[k][1][m:n].sum())
    if secondsBool:
        return np.array(tIndex)*60.0, np.array(intens)
    else:
        return np.array(tIndex), np.array(intens)

This function performs all the necessary steps to perform a Fourier Transform on experimental data.

In [11]:
def DFT_data(time, amp, window='hanning', padLen = 0.5, minBool=False, windowBool = True, padBool = False):
    '''
    Performs necessary manipulations to carry out a discrete Fourier Transform on experimental data.
    
    Parameters
    ------------
    time  :  can either be in sec, or min - if min, minBool must be True
    amp   :  amplitude spectrum of interferogram
    
    window: (optional, defaults to 'hanning' window)
             type of window to apply to the interpolated signal - available windows shown in 'windic' below
             
    padLen: (optional, defaults to 0.5)
             decimal value to determine front-end zero padding length based on original signals length
             --> e.g. '0.5' will give a pad that's 50% of the original signals length.
    
    minBool    : (defaults to False) True if signal is in minutes, otherwise it's assumed signal is in seconds
    windowBool : (defaults to True)  if windowing function is to be applied to signal
    padBool    : (defaults to False) if front-end zero padding is to be applied
    
    Returns
    ------------
    X  : Frequency axis
    Y  : Amplitude array of new frequency domain signal
    
    Requires UnivariateSpline (scipy), scipy.signal, and numpy to be imported.
    '''
    
    if minBool:
        time *= 60
        
    #dictionary for different window functions
    windic = {'bartlett': np.bartlett, 'blackman': np.blackman, 
              'hamming': np.hamming, 'hanning': np.hanning,
              'barthann': signal.barthann, 'bohman': signal.bohman, 'nuttall': signal.nuttall,
              'parzen': signal.parzen, 'tukey': signal.tukey}

    
    #picking length of new time vector based on original number of points
    
    n = int(len(time))*4 #choice of multiplying by 4 is somewhat arbitrary
    x_val = np.linspace(time[0], time[-1], n)
    
    #interpolating signal, smoothing value s=0 required for true interpolation
    ftspl = UnivariateSpline(time, amp, s=0)
    FTspl = ftspl(x_val)
    

    #---recovering y-axis---
    #where a windowing function is or isn't applied
    if windowBool:
        wind = windic[window](len(FTspl))
        wY = FTspl * wind
    else:
        wY = FTspl
    
    
    if padBool:
        wY = zero_padding(wY, padLen)
#         print('pad', len(FTspl))
#     plt.plot(wY)
    
    
    #---creating x-axis vector---
    N = int(len(wY)/2)
    mFac = 2/n #normalization factor
    W = np.fft.fftfreq(len(wY), np.diff(x_val).mean())
    X = W[:N]
    
    
    FY = np.fft.fft(wY)
    Y = np.abs(FY[:N]) * mFac
    
    
#     print(len(X), len(Y))
    return X, Y**2

This is essentially the same function as my **DFT_data** function. The difference here is it returns a modified time domain signal based on the different arguments and boolean options. This is mainly for interaction with widgets further down in the notebook. Did this as a quick check to my work further down, I plan on finding a more elegant solution; I didn't think adding more boolean arguments to **DFT_data** function would've looked good.

In [12]:
def Mod_Intf(time, amp, window='hanning', padLen = 0.5, minBool=False, windowBool = True, padBool = False):
    '''
    Performs necessary manipulations to carry out a discrete Fourier Transform on experimental data.
    
    Parameters
    ------------
    time  :  can either be in sec, or min - if min, minBool must be True
    amp   :  amplitude spectrum of interferogram
    
    window: (optional, defaults to 'hanning' window)
             type of window to apply to the interpolated signal - available windows shown in 'windic' below
             
    padLen: (optional, defaults to 0.5)
             decimal value to determine front-end zero padding length based on original signals length
             --> e.g. '0.5' will give a pad that's 50% of the original signals length.
    
    minBool    : (defaults to False) True if signal is in minutes, otherwise it's assumed signal is in seconds
    windowBool : (defaults to True)  if windowing function is to be applied to signal
    padBool    : (defaults to False) if front-end zero padding is to be applied
    
    Returns
    ------------
    X  : Frequency axis
    Y  : Amplitude array of new frequency domain signal
    
    Requires UnivariateSpline (scipy), scipy.signal, and numpy to be imported.
    '''
    
    if minBool:
        time *= 60
        
    #dictionary for different window functions
    windic = {'bartlett': np.bartlett, 'blackman': np.blackman, 
              'hamming': np.hamming, 'hanning': np.hanning,
              'barthann': signal.barthann, 'bohman': signal.bohman, 'nuttall': signal.nuttall,
              'parzen': signal.parzen, 'tukey': signal.tukey}

    
    #picking length of new time vector based on original number of points
    
    n = int(len(time))*4 #choice of multiplying by 4 is somewhat arbitrary
    x_val = np.linspace(time[0], time[-1], n)
    
    #interpolating signal, smoothing value s=0 required for true interpolation
    ftspl = UnivariateSpline(time, amp, s=0)
    FTspl = ftspl(x_val)
    

    #---recovering y-axis---
    #where a windowing function is or isn't applied
    if windowBool:
        wind = windic[window](len(FTspl))
        wY = FTspl * wind
    else:
        wY = FTspl
        
    if padBool:
        wY = zero_padding(wY, padLen)
#         print('pad', len(FTspl))

    return wY

In [13]:
def normVector(npArray, retFactor = False):
    '''
    Normalize any input vector.
    Drive maximum to 1.0
    '''
    if retFactor:
        return npArray/npArray.max(), npArray.max()
    else:
        return npArray/npArray.max()

## Multipeak Fitting Code

In [14]:
def gaussian(x, height, center, width, offset = 0):
    '''
    Returns a gaussian peak based upon the input params
    '''
    return height*np.exp(-(x - center)**2/(2*width**2)) + offset

In [15]:
def func(x, *params):#Building the hyothetical spectrum from our peak picking routine
    ''' *params of the form [center, amplitude, width ...] '''
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):# why 3? Count by 3s so we can grab amplitude, width, and centroid
        ctr = params[i]
        amp = params[i+1]
        wid = params[i+2]
        y = y + gaussian(x, amp, ctr, wid)
    return y

In [16]:
def fit_gaussians(guess, func, x, y):
    '''
    Uses scipy curve_fit to optimise the gaussian fitting
    '''
    popt, pcov = optimize.curve_fit(func, x, y, p0=guess, maxfev=10000)
    print('popt:', popt)
    fit = func(x, *popt)
    return (popt, fit, pcov)

In [17]:
def reportPeakParams(a, w, c, peakNum = ''):
    fwhm = w*2.35482 #in recognition that the width of a gaussian is not FWHM
    rp = (1.0*c)/fwhm
    c*=1000
    w*=1000
    retStr = "Peak %s @ %.3f ms, Rp = %.2f"%(peakNum, c, rp)
    print(retStr)
    return rp, fwhm, retStr

In [18]:
def findYPeaks(y, mph = 2, mpd = 70, mpw = 5, verbose = False):
    '''
    minimum peak height
    minimum peak distance
    minimum peak width
    '''
    indexes, properties, = signal.find_peaks(y, distance = mpd, width = mpw, height = mph)
    if verbose:
        print('Peaks found at the following indices: %s' % (indexes))
    return indexes

In [19]:
def makeGuess(x, y, peakInd, peakWidth = 0.0001):
    '''
    "guess" is a list that contains centers, amplitude, and widths
    '''
    if len(peakInd)<1:
        return False
    numPeaks = len(peakInd)
    amplitudes = y[peakInd]
    widths = np.zeros(numPeaks) + peakWidth
    centers = x[peakInd]*1.0#ensure it is a float
    
    guess = []

    for i in range(numPeaks):
        guess.append(centers[i])
        guess.append(amplitudes[i])
        guess.append(widths[i])
        
    guessSpec = np.zeros(len(y))#dummy spectrum
#     xArray = np.arange(len(y))#xarray of dummy spectrum
    # Create spectrum
    for a, c, w in zip(amplitudes, centers, widths):
        guessSpec += gaussian(x, a, c, w)
        
    return guess, guessSpec

In [20]:
def fitPeaksMP(x, y, guess):
    '''
    Attempts a multipeak fit based upon the input guesses.  
    Make sure the baseline is close to zero or this will most likely fail.     
    '''
    params, fit, pcov = fit_gaussians(guess, func, x, y) ###params is the array of gaussian guesses
    return params, fit, pcov

In [21]:
def results2df(fitResult):
    '''
    Returns a dictionary of results to a dataframe
    '''
    dataDict = {}
    dataDict['Peak #'] = []
    dataDict['Centroid'] = []
    dataDict['FWHM'] = []
    dataDict['Amplitude'] = []
    dataDict['Rp'] = []
    for i,j in enumerate(range(0, len(fitResult), 3)): 
        ctr = fitResult[j] 
        amp = fitResult[j+1]
        width = fitResult[j+2]
        rp, fwhm, retStr = reportPeakParams(amp, width, ctr, peakNum = i+1)
        dataDict['Peak #'].append(i+1)
        dataDict['Centroid'].append(ctr)
        dataDict['FWHM'].append(fwhm)
        dataDict['Amplitude'].append(amp)
        dataDict['Rp'].append(rp)
    
#     df = pd.DataFrame.from_dict(dataDict)
    
    
    df = pd.DataFrame(dataDict, columns=['Peak #', 'Centroid', 'FWHM', 'Amplitude', 'Rp'])  
    df.set_index('Peak #', inplace=True)
    
    return df

In [22]:
def fitPeaksWorkflow(xVec, yVec, peakInd, peakProperties, fitFactor = 4, useGaussian = True, debug = False):
    '''
    
    Pass in the peak indices
    Normalize and get scale factor
    Scale Data upon return
    
    
    Need to figure out a way to detect things going off the rails.
    
    '''

    yVecN, scaleFactor = normVector(yVec, retFactor = True)
    
    if len(peakInd) > 0:
        widthGuess = np.array(peakProperties['widths']).mean()/(fitFactor*1.0)
        xDiff = np.diff(xVec).mean()
        widthGuess = xDiff*widthGuess
        guess, guessSpec = makeGuess(xVec, yVecN, peakInd, peakWidth = widthGuess)
        params, fit, pcov =  fitPeaksMP(xVec, yVecN, guess)
        
        curDF = results2df(params)
        fitResults = getResultsSummary(xVec, params, scaleFactor, fitFactor)
        return curDF, fitResults, params, scaleFactor
    else:
        print("No Peaks Found")
        return [None, None, None, None]

In [23]:
def getResultsSummary(xVec, peakParams, scaleFactor, widthFactor = 4):
    fitResults = {}
    fitResults['XARRAYS'] = []
    fitResults['YARRAYS'] = []
    fitResults['CENTROIDS'] = []
    fitResults['WIDTHS'] = []
    fitResults['AMPLITUDES'] = []
    fitResults['SCALEFACTORS'] = []
    
    peakProfiles = []
    for i,j in enumerate(range(0, len(peakParams), 3)): 
        ctr = peakParams[j]
        fitResults['CENTROIDS'].append(ctr)
        
        amp = peakParams[j+1]
        fitResults['AMPLITUDES'].append(amp)
        
        width = peakParams[j+2]
        fitResults['WIDTHS'].append(width)
        
        widthTol = width*widthFactor #get the peak width and scale
        xMin = ctr-widthTol
        xMax = ctr+widthTol
        
        idxMin = (np.abs(xVec - xMin)).argmin()
        idxMax = (np.abs(xVec - xMax)).argmin()
        
        fitResults['XARRAYS'].append(xVec[idxMin:idxMax])
        yVec = gaussian(xVec, amp, ctr, width)*scaleFactor
        fitResults['YARRAYS'].append(yVec[idxMin:idxMax])
        
        fitResults['SCALEFACTORS'].append(scaleFactor)
        
    return fitResults

Quick mobility calculation

In [24]:
def moCal(dtime, length = 17.385, voltage = 7860, T = 297.7, P = 690):
    '''
    Returns reduced mobility.
    
    Parameters
    ------------------
    dtime  : Drift time in (ms)
    length : Drift tube length (cm)
    voltage: Voltage across drift tube (V)
    T      : Temperature (K)
    P      : Pressure (torr)
    '''
    v = length / (dtime / 1000) #cm/s
    E = voltage / length # V/cm
    K = v / E
    K0 = K * (P/760)*(273.15/T)
    
    return K0

In [25]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 2CEA-1333

 Directory of C:\Users\bhclo\Desktop\GoogleDrive\My Drive\SPELLS\Panel

11/24/2021  07:29 PM    <DIR>          .
02/08/2015  11:07 AM    <DIR>          ..
11/24/2021  07:29 PM    <DIR>          .ipynb_checkpoints
06/30/2021  01:10 PM    <DIR>          __pycache__
02/12/2021  06:31 PM       144,246,924 02042021_IMS_Mb_dn_3st_10uM_5_5k_5min_10IIT_60k_2.RAW
07/03/2021  08:26 PM        30,006,672 210316_250nM_CRP_200AmA_6-0scans_5-5005Hz_480s_1730He_236C_1.mzML
07/14/2021  08:31 PM         9,320,491 210316_250nM_CRP_200AmA_6-0scans_5-5005Hz_480s_1730He_236C_1.RAW
06/30/2021  10:47 AM             4,183 attractors.ipynb
07/05/2021  07:19 PM    <DIR>          Digilent Processor
11/21/2021  04:26 PM           156,141 FT-IM Processing v0.9.ipynb
11/24/2021  07:29 PM           189,382 FT-IM Processing v0.95.ipynb
06/30/2021  10:53 AM            30,463 Introduction.ipynb
06/30/2021  10:51 AM           141,566 IPyWidget.ipynb
06/

In [26]:
# fileType = '*.mzML'
# path = os.path.join(os.getcwd(),fileType)
# fileNames = []
# for fname in glob.glob(path):
#     fileNames.append(fname)
#     #print(fname)

In [27]:
# fileNames.sort()

In [28]:
# fileNames

In [29]:
# curFile = fileNames[0]

In [30]:
# spectral_data = [s for s in mzml.read(curFile)]

In [31]:
# mzLo = spectral_data[0]['scanList']['scan'][0]['scanWindowList']['scanWindow'][0]['scan window lower limit'].real
# mzHi = spectral_data[0]['scanList']['scan'][0]['scanWindowList']['scanWindow'][0]['scan window upper limit'].real
# # mzHi = spectral_data[0]['scanList']['scan'][0]['scanWindowList']['scanWindow']['scan window upper limit']
# print(mzLo, mzHi)

In [32]:
# len(spectral_data)

In [33]:
# s = spectral_data[0]

In [34]:
# s['scanList']['scan'][0]['scan start time'].real*60

In [35]:
# s['m/z array']

In [36]:
# np.column_stack((s['m/z array'], s['intensity array']))

In [37]:
# with mzml.read(curFile) as reader:
#     auxiliary.print_tree(next(reader))

In [38]:
# x, y, rawData = loadOrbiMS_v2(curFile, 0.05)

In [39]:
# plt.plot(x,y)

## GUI Widgets

In [40]:
pn.widgets.FileSelector?

In [41]:
class FILEBOX(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["column"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)
        #Note that if the directory is not nested at least two down this can get ugly. Change the root_directory to something appropriate
        
        #https://github.com/holoviz/panel/issues/2051
        self.fileSelector = pn.widgets.FileSelector(directory = os.getcwd(), root_directory = '../../', height = 350)
        self.orbiBool = pn.widgets.Checkbox(name='Orbitrap Data')
        self.loadButton = pn.widgets.Button(name='Load Data File', button_type='primary')        
        self.label = pn.pane.Markdown("""## *Go Cougs!*""")
        
        self[:] = [pn.layout.Divider(margin=(-20, 0, 0, 0)),self.fileSelector, self.orbiBool, self.loadButton, self.label]

fb = FILEBOX()
fb

FILEBOX
    [0] Divider(margin=(-20, 0, 0, 0))
    [1] FileSelector(height=350, root_directory='C:\\Users\\bhclo\\Desktop...)
    [2] Checkbox(name='Orbitrap Data')
    [3] Button(button_type='primary', name='Load Data File')
    [4] Markdown(str)

In [42]:
fb.fileSelector.value

[]

In [43]:
class FILEACCORD(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, fileBox, **params):
        self._rename["column"] = None
        self.fileBox = fileBox
        self.fileWidget = pn.Column(self.fileBox)
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)
   
        self.fileAccord = pn.Accordion(("File Selection", self.fileWidget))
        self[:] = [self.fileAccord]


fa = FILEACCORD(fb)
fa


FILEACCORD
    [0] Accordion
        [0] Column
            [0] FILEBOX
                [0] Divider(margin=(-20, 0, 0, 0))
                [1] FileSelector(height=350, root_directory='C:\\Users\\bhclo\\Desktop...)
                [2] Checkbox(name='Orbitrap Data')
                [3] Button(button_type='primary', name='Load Data File')
                [4] Markdown(str)

In [44]:
class EXTRACTBOX(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["column"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)

        self.lowerXInput = pn.widgets.FloatInput(name='Lower Bound', value=5., step=1e-1, start=0, end=1000000)
        self.upperXInput = pn.widgets.FloatInput(name='Upper Bound', value=500., step=1e-1, start=1, end=1000000)
        self.xButton = pn.widgets.Button(name='Update x-axis', button_type='default')
        self.xicButton = pn.widgets.Button(name='Extract XIC', button_type='primary')
        
        self[:] = [pn.layout.Divider(margin=(-20, 0, 0, 0)),
                   pn.Row(self.lowerXInput,
                          self.upperXInput,
                          self.xButton,
                          self.xicButton
                         )
                  ]

eb = EXTRACTBOX()
eb

EXTRACTBOX
    [0] Divider(margin=(-20, 0, 0, 0))
    [1] Row
        [0] FloatInput(end=1000000, name='Lower Bound', start=0, value=5.0)
        [1] FloatInput(end=1000000, name='Upper Bound', start=1, value=500.0)
        [2] Button(name='Update x-axis')
        [3] Button(button_type='primary', name='Extract XIC')

In [45]:
eb.upperXInput.value

500.0

In [46]:
def dump2csv(xVector, yVector, fileName, listInfo = []):
    '''
    Assumes the filename loaded will be split to get the core, basename of the file. The listInfo will be split and
    each entry added to the file name end. From there the x,y will be dumped to csv.
    BEWARE: This overwrites.  No checks. 
    '''
    
    coreName = fileName.split('.')[0]
    for i in listInfo:#go up to the last element
        coreName += '_'
        coreName += i
    coreName += '.csv'
    
    df = pd.DataFrame({"X" : xVector, "Y" : yVector})
    df.to_csv(coreName, index=False)        

In [47]:
def dumpPlotImage(fig, fileName, listInfo = [], fileFormat = PLOTOUTPUT):
    '''
    Assumes the filename loaded will be split to get the core, basename of the file. The listInfo will be split and
    each entry added to the file name end. From there the figure image will be dumped to the user specified file formate.
    BEWARE: This overwrites.  No checks. 
    '''
    
    coreName = fileName.split('.')[0]
    for i in listInfo:#go up to the last element
        coreName += '_'
        coreName += i
    coreName += fileFormat
    
    fig.write_image(coreName, engine="kaleido")

In [48]:
class SAVEBOX(pn.Row):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["row"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)

        self.saveVectorButton = pn.widgets.Button(name='Save CSV', button_type='warning')
        self.savePlotButton = pn.widgets.Button(name='Save PNG', button_type='primary')

        self[:] = [self.saveVectorButton, self.savePlotButton]

sb = SAVEBOX()
sb

SAVEBOX
    [0] Button(button_type='warning', name='Save CSV')
    [1] Button(button_type='primary', name='Save PNG')

In [49]:
class SAVEACCORD(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["column"] = None

        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)
        
        self.saveVectorButton = pn.widgets.Button(name='Save CSV', button_type='warning')
        self.savePlotButton = pn.widgets.Button(name='Save PNG', button_type='primary')
        
        self.saveRow = pn.Row(self.saveVectorButton, self.savePlotButton)
        
        self.plotList = ['ATD', 'Modulated Signal', 'Mass Spectrum']

        self.plotSelector = pn.widgets.Select(name='Data Selection', options=self.plotList)
        
        
        self.saveWidget = pn.Column(self.plotSelector,self.saveRow)

        self.saveAccord = pn.Accordion(("Data Export", self.saveWidget))
        self[:] = [self.saveAccord]


sa = SAVEACCORD()
sa


SAVEACCORD
    [0] Accordion
        [0] Column
            [0] Select(name='Data Selection', options=['ATD', 'Modulated Signal'...], value='ATD')
            [1] Row
                [0] Button(button_type='warning', name='Save CSV')
                [1] Button(button_type='primary', name='Save PNG')

In [50]:
sa.plotSelector.options

['ATD', 'Modulated Signal', 'Mass Spectrum']

In [51]:
sa.plotSelector.value

'ATD'

In [52]:
class PROCESSINGBOX(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["column"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)

        self.windDict = {'bartlett': np.bartlett, 'blackman': np.blackman, 
              'hamming': np.hamming, 'hanning': np.hanning,
              'barthann': signal.barthann, 'bohman': signal.bohman, 'nuttall': signal.nuttall,
              'parzen': signal.parzen, 'tukey': signal.tukey}

        self.windowCB = pn.widgets.Checkbox(name='Window')
        self.windowTypeSelector = pn.widgets.Select(name='Window Type', options=list(self.windDict.keys()))
        self.ftButton = pn.widgets.Button(name='Apply FT', button_type='danger')

        self.zeroPadCB = pn.widgets.Checkbox(name='Zero pad')
        self.zeroSlider = pn.widgets.FloatInput(name='Pad Length', value=0.5, step=0.1, start=0, end=2)
        self.resetButton = pn.widgets.Button(name='Reset Plot', button_type='warning')

        self.processRow1 = pn.Row(self.windowCB, self.windowTypeSelector, self.ftButton)
        self.processRow2 = pn.Row(self.zeroPadCB, self.zeroSlider, self.resetButton)
        
        
        self.pointCutoffInput = pn.widgets.IntInput(name='Point Cutoff', value=DCCUTOFF, step=1, start=1, end=10000)
        
        
        self.mainBox = pn.WidgetBox('#### Processing Control', self.processRow1, self.processRow2, self.pointCutoffInput)
        
        self[:] = [self.mainBox]

#         processingBox = pn.WidgetBox('#### Processing Control', processRow1, processRow2)
#         processingBox

pb = PROCESSINGBOX()
pb

PROCESSINGBOX
    [0] WidgetBox
        [0] Markdown(str)
        [1] Row
            [0] Checkbox(name='Window')
            [1] Select(name='Window Type', options=['bartlett', 'blackman', ...], value='bartlett')
            [2] Button(button_type='danger', name='Apply FT')
        [2] Row
            [0] Checkbox(name='Zero pad')
            [1] FloatInput(end=2, name='Pad Length', start=0, value=0.5)
            [2] Button(button_type='warning', name='Reset Plot')
        [3] IntInput(end=10000, name='Point Cutoff', start=1, value=50)

In [53]:
pb.zeroPadCB.value

False

In [54]:
pb.zeroSlider.value

0.5

In [55]:
def convertHztoATD(hzVector, sweepRate, convert2ms = True):
    atdVec = hzVector/sweepRate
    if convert2ms:
        atdVec/=1000.0 #convert to ms. 
        
    return atdVec


In [56]:
class ATDBOX(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["column"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)


        self.paramKeys = ['Freq. Sweep (Hz/s)', 
                     'Drift length (cm)', 
                     'Drift tube voltage (V)', 
                     'Temperature (K)', 
                     'Pressure (torr)']

        
        #The following values are read from the Global Variables defined at the start of the notebook
#         FREQSWEEP = 15 #Hz/s
#         DRIFTLENGTH = 100 #cm
#         DRIFTVOLTAGE = 1800 #Volts
#         TEMPERATURE = 298 #Temp (K)
#         PRESSURE = 3.5
        self.startVals = [FREQSWEEP, 
                     DRIFTLENGTH, 
                     DRIFTVOLTAGE, 
                     TEMPERATURE, 
                     PRESSURE]

        self.steps = [0.01,
                0.01,
                0.1,
                0.01,
                0.01]

        self.starts = [0,
                0,
                0,
                0,
                0]

        self.ends = [1000,
              2000,
              50000,
              2000,
              2000]

        
        #Ideally this could be done with a loop but that was tried with the Panel input system and it failed.
        i = 0
        self.sweepRateWidg = pn.widgets.FloatInput(name=self.paramKeys[i], 
                                                   value=self.startVals[i], 
                                                   step=self.steps[i], 
                                                   start=self.starts[0], 
                                                   end=self.ends[i])
        i+=1
        
        self.driftLenWidg = pn.widgets.FloatInput(name=self.paramKeys[i], 
                                                   value=self.startVals[i], 
                                                   step=self.steps[i], 
                                                   start=self.starts[0], 
                                                   end=self.ends[i])
        i+=1
        
        self.driftVWidg = pn.widgets.FloatInput(name=self.paramKeys[i], 
                                                   value=self.startVals[i], 
                                                   step=self.steps[i], 
                                                   start=self.starts[0], 
                                                   end=self.ends[i])
        i+=1
        
        self.tempWidg = pn.widgets.FloatInput(name=self.paramKeys[i], 
                                                   value=self.startVals[i], 
                                                   step=self.steps[i], 
                                                   start=self.starts[0], 
                                                   end=self.ends[i])
        i+=1
        
        self.pressWidg = pn.widgets.FloatInput(name=self.paramKeys[i], 
                                                   value=self.startVals[i], 
                                                   step=self.steps[i], 
                                                   start=self.starts[0], 
                                                   end=self.ends[i])
        

           
        self.magSqBool = pn.widgets.Checkbox(name='Magnitude Squared')
        self.magSqBool.value = True

        self.ATDButton = pn.widgets.Button(name='Convert to ATD', button_type='warning')

        self.outputRow = pn.Row(self.magSqBool, self.ATDButton)        
        
        self.atdWidget = pn.Column(
            pn.layout.Divider(margin=(-20, 0, 0, 0)),
            self.sweepRateWidg,
            self.driftLenWidg,
            self.driftVWidg,
            self.tempWidg,
            self.pressWidg,
            self.outputRow
        )

        self.atdAccord = pn.Accordion(("ATD Control", self.atdWidget))
        self[:] = [self.atdAccord]


ab = ATDBOX()
ab


ATDBOX
    [0] Accordion
        [0] Column
            [0] Divider(margin=(-20, 0, 0, 0))
            [1] FloatInput(end=1000, name='Freq. Sweep (Hz/s)', start=0, step=0.01, value=15)
            [2] FloatInput(end=2000, name='Drift length (cm)', start=0, step=0.01, value=100)
            [3] FloatInput(end=50000, name='Drift tube voltage (V)', start=0, value=1800)
            [4] FloatInput(end=2000, name='Temperature (K)', start=0, step=0.01, value=298)
            [5] FloatInput(end=2000, name='Pressure (torr)', start=0, step=0.01, value=3.5)
            [6] Row
                [0] Checkbox(name='Magnitude Squared', value=True)
                [1] Button(button_type='warning', name='Convert to ATD')

In [57]:
class PEAKBOX(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, startMD = 20, startMW = 0.1, startMH = 0.25, startOrder = 21, startPoly = 3, startWF = 4, lowerStartVal = 0.01, upperStartVal = 2, **params):
        self._rename["column"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)
        
        

        self.distanceSlider = pn.widgets.FloatInput(name='Minimum Distance', start=1, end=500000, step=1, value=startMD)
        self.widthSlider = pn.widgets.FloatInput(name='Minimum Width', start=0.000001, end=1000000, step=0.1, value=startMW)
        self.minHSlider = pn.widgets.FloatInput(name='Minimum Height', start=0.001, end=1, step=0.05, value=startMH, format='1[.]00')
        self.orderSlider = pn.widgets.IntInput(name='Smooth Window', start=3, end=30001, step=2, value=startOrder)
        self.polySlider = pn.widgets.IntInput(name='Polynomial Order', start=3, end=11, step=1, value=startPoly)
        self.peakWinSlider = pn.widgets.IntInput(name='Fit Width Factor', start=3, end=20, step=1, value=startWF)
              
        self.smoothButton = pn.widgets.Button(name='Smooth Data', button_type='warning')
        self.resetSmoothButton = pn.widgets.Button(name='Undo Smooth', button_type='primary')
        
        self.buttonRow = pn.Row(self.smoothButton, self.resetSmoothButton)

        self.corrOffsetBool = pn.widgets.Checkbox(name='Autocorrect Y Offset')
        self.corrOffsetBool.value = True        

        self.lowerXNoise = pn.widgets.FloatInput(name='Lower Noise X', value=lowerStartVal, step=1e-1, start=0, end=1000000)
        self.upperXNoise = pn.widgets.FloatInput(name='Upper Noise X', value=upperStartVal, step=1e-1, start=0.0001, end=1000000)
        
        self.noiseRow = pn.Row(self.lowerXNoise, self.upperXNoise)
        self.estimateButton = pn.widgets.Button(name='Estimate Offset', button_type='danger')
        self.corrButton = pn.widgets.Button(name='Correct Offset', button_type='success')
        self.corrRow = pn.Row(self.estimateButton, self.corrButton)
        

        self.peakWidget = pn.Column(
            pn.layout.Divider(margin=(-20, 0, 0, 0)),
            self.distanceSlider,
            self.widthSlider,
            self.minHSlider,
            self.peakWinSlider,
            self.orderSlider,
            self.polySlider,
            self.buttonRow,
            self.corrOffsetBool,
            self.noiseRow,
            self.corrRow
        
        )


        self.peakAccord = pn.Accordion(("Peak Parameters", self.peakWidget))

        self[:] = [self.peakAccord]


peakbox = PEAKBOX()
peakbox


PEAKBOX
    [0] Accordion
        [0] Column
            [0] Divider(margin=(-20, 0, 0, 0))
            [1] FloatInput(end=500000, name='Minimum Distance', start=1, step=1, value=20)
            [2] FloatInput(end=1000000, name='Minimum Width', start=1e-06, value=0.1)
            [3] FloatInput(end=1, format='1[.]00', name='Minimum Height', start=0.001, step=0.05, value=0.25)
            [4] IntInput(end=20, name='Fit Width Factor', start=3, value=4)
            [5] IntInput(end=30001, name='Smooth Window', start=3, step=2, value=21)
            [6] IntInput(end=11, name='Polynomial Order', start=3, value=3)
            [7] Row
                [0] Button(button_type='warning', name='Smooth Data')
                [1] Button(button_type='primary', name='Undo Smooth')
            [8] Checkbox(name='Autocorrect Y Offset', value=True)
            [9] Row
                [0] FloatInput(end=1000000, name='Lower Noise X', start=0, value=0.01)
                [1] FloatInput(end=1000000, name='Upper Noise X', start=0.0001, value=2)
            [10] Row
                [0] Button(button_type='danger', name='Estimate Offset')
                [1] Button(button_type='success', name='Correct Offset')

In [58]:
class SPECTRALHANDLER():
    '''
    Requires scipy.signal
    
    TODO: Add an export option
    '''
    def __init__(self):
        
        self.clearData()
    
    def clearData(self):
        self.dataBool = False
        self.fitBool = False
        self.dataSet = None
        self.peakBool = None
        self.peakInds = []
        self.peakProperties = None
        self.scaleFactor = 1
        self.fitResults = None
        self.resultDF = None
        
        
        self.offsetValue = 0
        self.xNoisePntMin = 0
        self.xNoisePntMax = 1#just so this one is larger than the other. 
        
    
    def estimateNoise(self, percMin = 0.80, percMax = 0.95):
        '''
        Assumes it is the last ~15% of the spectrum. 
        This assumption may be entirely INVALID
        '''
        if self.dataBool:
            lenSpec = len(self.dataSet)
            self.xNoisePntMin = np.int(lenSpec*percMin)
            self.xNoisePntMax = np.int(lenSpec*percMax)
            self.offsetValue = getYOffset(self.dataSet, self.xNoisePntMin, self.xNoisePntMax)
    
    def updateOffset(self, pntStart, pntEnd):
        if self.dataBool:
            self.xNoisePntMin = pntStart
            self.xNoisePntMax = pntEnd
            self.offsetValue = getYOffset(self.dataSet, self.xNoisePntMin, self.xNoisePntMax)
    
    def correctOffset(self):
        if self.dataBool:
            self.dataSet = self.dataSet-self.offsetValue
    
    def setResultDF(self, resultDF):
        '''
        Sets the peak fit results to a dataframe
        '''
#         resultDF.style.hide_index()
        self.resultDF = resultDF
    
    def setResults(self, fitResults):
        '''
        Puts peak fit results into memory
        '''
        self.fitResults = fitResults
        self.fitBool = True

    def setData(self, dataVector):    
        self.dataSet = dataVector
        self.dataBool = True
        self.peakBool = False
        self.fitBool = False
    
    def setPeakInd(self, peakLocs):
        self.peakInds = peakLocs
        self.peakBool = True

    def smoothDataSet(self, order, poly):
        if self.dataBool:
            self.smthResult = signal.savgol_filter(self.dataSet, order, poly)
            return self.smthResult
        
    def getPeakLocs(self, curD, curW, curH):
        '''
        curD = peak Distance
        curW = peak Width
        curH = peak Heigh
        '''
        if self.dataBool:
            peakLocs, properties, = signal.find_peaks(self.normVector(self.dataSet), distance = curD, width = curW, height = curH)#, rel_height = 0.5)
            self.setPeakInd(peakLocs)
            self.peakProperties = properties
        else:
            print("No data have been specified, call `setData(yournumpyarray)")

            
    def normVector(self, retFactor = False):
        '''
        Normalize any input vector.
        Drive maximum to 1.0
        '''
        if self.dataBool:
            self.scaleFactor = self.dataSet.max()#get maximum value
            return self.dataSet/self.dataSet.max()            
            
    def performFits(self):
        print("Starting Fits")     

In [59]:
sh = SPECTRALHANDLER()

In [60]:
sh.dataBool

False

In [61]:
class FINDPEAKSBOX(pn.Column):
    '''
    Adapted from: https://discourse.holoviz.org/t/how-to-create-a-self-contained-custom-panel/985/6
    '''
    def __init__(self, **params):
        self._rename["column"] = None
        # Note:
        # I need to add _rename of `column` in order to not get the exception
        # AttributeError: unexpected attribute 'column' to Column, possible attributes are align, aspect_ratio, background, children, css_classes, disabled, height, height_policy, js_event_callbacks, js_property_callbacks, margin, max_height, max_width, min_height, min_width, name, rows, sizing_mode, spacing, subscribed_events, tags, visible, width or width_policy
        # The reason is that the code of Column tries to map every parameter on the class to a property on the underlying bokeh model.
        # and `column` is not a property on the underlying bokeh model.

        super().__init__(**params)

        self.findButton = pn.widgets.Button(name='Find Peaks', button_type='success')
        self.fitButton = pn.widgets.Button(name='Fit Peaks', button_type='danger')
        self.resetATDButton = pn.widgets.Button(name='Reset ATD Plot', button_type='primary')
      
        self[:] = [pn.layout.Divider(margin=(-20, 0, 0, 0)),
                   pn.Row(self.findButton,
                          self.fitButton
                         ),
                   self.resetATDButton
                  ]

   
findbox = FINDPEAKSBOX()
findbox

FINDPEAKSBOX
    [0] Divider(margin=(-20, 0, 0, 0))
    [1] Row
        [0] Button(button_type='success', name='Find Peaks')
        [1] Button(button_type='danger', name='Fit Peaks')
    [2] Button(button_type='primary', name='Reset ATD Plot')

In [62]:
class DataPlotter:
    def __init__(self, **params):#, df, rows=1, cols=1, legend=dict(x=0.77,y=1)):
        
        '''
        setting self.fig.data = [] clears the plot
        
        '''
        
        #redundant. I know. Helps me see things clearly--or so I think.
        #########################
        self.atdAccord = ATDBOX()
        self.sweepRateWidg = self.atdAccord.sweepRateWidg
        self.driftLenWidg = self.atdAccord.driftLenWidg
        self.tempWidg = self.atdAccord.tempWidg
        self.pressWidg = self.atdAccord.pressWidg
        self.ATDButton = self.atdAccord.ATDButton      
        
        self.controlDict = {}
        self.controlDict['Length (cm)'] = self.driftLenWidg
        self.controlDict['Voltage (V)'] = self.driftLenWidg
        self.controlDict['Temperature (K)'] = self.tempWidg
        self.controlDict['Pressure (torr)'] = self.pressWidg      
        
        self.ATDButton.on_click(self.ATDButtonClick)
        
        
        
        #########################
        self.peakAccord = PEAKBOX()
        
        self.distanceSlider = self.peakAccord.distanceSlider
        self.widthSlider = self.peakAccord.widthSlider
        self.minHSlider = self.peakAccord.minHSlider
        self.orderSlider = self.peakAccord.orderSlider
        self.polySlider = self.peakAccord.polySlider
        self.peakWinSlider = self.peakAccord.peakWinSlider
        
        self.lowerXNoise = self.peakAccord.lowerXNoise
        self.upperXNoise = self.peakAccord.upperXNoise
        
        
#         self.peakAccord.smoothButton.on_click(self.smoothDataSet)
#         self.peakAccord.resetSmoothButton.on_click(self.resetSmooth)
        
#         self.peakAccord.estimateButton.on_click(self.estimateBaselinePoints)
#         self.peakAccord.corrButton.on_click(self.correctBaselineOffset)       
        
        
        #########################
        self.extractBox = EXTRACTBOX()
        
        #redundant. I know. Helps me see things clearly--or so I think. 
        self.extractButton = self.extractBox.xicButton
        self.updateXButton = self.extractBox.xButton
        
        self.lowerXInput = self.extractBox.lowerXInput
        self.upperXInput = self.extractBox.upperXInput
        
        self.updateXButton.on_click(self.updateAxisButtonClick)
        self.extractButton.on_click(self.extractButtonClick)
        
        #########################
        self.processingBox = PROCESSINGBOX()
        
        self.windowCB = self.processingBox.windowCB
        self.windowTypeSelector = self.processingBox.windowTypeSelector
        self.ftButton = self.processingBox.ftButton
        self.ftButton.on_click(self.applyFTButtonClick)
        
        self.zeroPadCB = self.processingBox.zeroPadCB
        self.padWidget = self.processingBox.zeroSlider
        self.resetPlotButton = self.processingBox.resetButton    
        
        self.pointCutoffInput = self.processingBox.pointCutoffInput
        
        
        #########################
        self.findPeakBox = FINDPEAKSBOX()
        self.peakHandler = SPECTRALHANDLER()
        

        self.findButton = self.findPeakBox.findButton
        self.findButton.on_click(self.findPButtonClick)
        
        self.fitButton = self.findPeakBox.fitButton
        self.fitButton.on_click(self.fitPButtonClick)
#         self.resetButton   
        self.findPeakBox.resetATDButton.on_click(self.resetATDButtonClick)
        
        #FIXME
        dfDict = {}
        dfDict['Peaks Not Set'] = []
        emptyDF = pd.DataFrame.from_dict(dfDict)
#         self.peakHandler.setResultDF(emptyDF)
        
        self.peaksDF = emptyDF#pd.util.testing.makeMixedDataFrame()
        
        #########################
        self.dataAccord = SAVEACCORD()
        self.saveVectorButton = self.dataAccord.saveVectorButton
        self.savePlotButton = self.dataAccord.savePlotButton
        
        #note that the link to the option and the plots is made below
#         self.plotList = ['ATD', 'Modulated Signal', 'Mass Spectrum']
        self.plotSelector = self.dataAccord.plotSelector
        self.savePlotButton.on_click(self.savePlotFigure)
        self.saveVectorButton.on_click(self.savePlotCSV)
        
        #########################
        self.fileBox = FILEBOX()
        self.fileSelector = self.fileBox.fileSelector
        self.orbiBool = self.fileBox.orbiBool
        self.loadButton = self.fileBox.loadButton
        
        self.loadButton.on_click(self.loadButtonClick)
        
        
        self.debugArea = pn.widgets.input.TextAreaInput(name='Debug Area', placeholder='TBD')
        self.fileName = None
        
        
        #Add some dummy data
        self.x = np.arange(20)#[]
        self.y = np.sin(self.x)#[]
        
        self.mzFig = go.Figure()
        self.mzPlot = go.Scatter(x=self.x, y=self.y, name = "Raw Data", line=dict(color='crimson', width=1.5))
        self.mzFig.add_trace(self.mzPlot)
        self.mzFig.update_layout(template = 'plotly_dark', autosize=True, height = 400,
                                 xaxis=dict(rangeslider = dict(visible=True), type = '-'))
        
        self.mzTrace = self.mzFig['data'][0]
        
        self.mzFig.update_xaxes(title='m/z')
        self.mzFig.update_yaxes(title='Intensity (a.u.)')        
        
        #setting up interferogram
        self.timePlot = go.Scatter(x=self.x, y=self.y, line=dict(color='teal', width=1.5))
        
        self.timeFig = go.Figure()#Widget(self.timeData)
        self.timeFig.add_trace(self.timePlot)
        self.timeFig.update_layout(title='', autosize=True, height=400, template='plotly_dark')
        
        self.timeData = self.timeFig.data[0]
        
        self.timeFig.update_xaxes(title='Time (s)')
        self.timeFig.update_yaxes(title='Intensity (a.u.)')
        
        #setting up mobility spectrum
        
        self.atdPlot = go.Scatter(x=self.x, y=self.y, name='Data', line=dict(color='goldenrod', width=1.5))
        
        self.atdFig = go.Figure()
        self.atdFig.add_trace(self.atdPlot)
        self.atdFig.update_layout(title='', autosize=True, template='plotly_dark')
        self.atdData = self.atdFig.data[0]
        
        
        self.atdFig.add_trace(go.Scatter(
            x=[],
            y=np.array([]),
                name="Peak Location",
                mode="markers",
                marker = dict(
                    color = 'rgb(255,255,255)',
                    size = 8,
                    line = dict(
                      color = 'rgb(255,0,0)',
                      width = 2
                    )
                  )
                )
        )
        
        
        self.peakTrace = self.atdFig.data[1]
        
               
        self.panes = []
        self.plotBox = []
        
        '''
        Other Data Containers
        '''
        self.intfX = np.arange(10) #dummy values
        self.intfY = np.arange(10)
        
        self.linkFigs2DataSelector()

    def linkFigs2DataSelector(self):
        dataOptions = self.plotSelector.options
        figList = [self.atdFig, self.timeFig, self.mzFig]
        saveTag = ['ATD', 'TIME', 'MZ']
        dataList = [self.atdData, self.timeData, self.mzTrace]
        self.plotDict = {}
        for i,op in enumerate(dataOptions):
            self.plotDict[op] = [figList[i], saveTag[i], dataList[i]]

    def savePlotCSV(self, event = None):
        '''
        FIXME -- Add try, except to make sure keys and items are aligned
        '''
        curSelection = self.plotSelector.value
        if curSelection in self.plotDict.keys():
            curData = self.plotDict[curSelection][2]
            curTag = self.plotDict[curSelection][1]
            if self.fileName != None:
                listInfo = []
#                 listInfo.append()
#                 listInfo.append('_')
                listInfo.append(curTag)
#                 dump2csv(xVector, yVector, fileName, listInfo = [])
                dump2csv(curData.x, curData.y, self.fileName, listInfo = listInfo)
                reportStr = 'Current %s csv saved.'%curSelection
                self.debugArea.value = str(reportStr)
                    
            
    def savePlotFigure(self, event = None):
        '''
        FIXME -- Add try, except to make sure keys and items are aligned
        '''
        curSelection = self.plotSelector.value
        if curSelection in self.plotDict.keys():
            curFig = self.plotDict[curSelection][0]
            curTag = self.plotDict[curSelection][1]
            if self.fileName != None:
                listInfo = []
#                 listInfo.append()
#                 listInfo.append('_')
                listInfo.append(curTag)
                dumpPlotImage(curFig, self.fileName, listInfo = listInfo)
                reportStr = 'Current %s figure saved.'%curSelection
                self.debugArea.value = str(reportStr)
                          
        
        
    def setPanes(self):
        '''
        This is needed to access the javascript info in plotly through panel
        Ugh: https://ask.csdn.net/questions/3032547
        '''
        self.mzPane = pn.pane.Plotly(self.mzFig)
        self.timePane = pn.pane.Plotly(self.timeFig)
        self.atdPane = pn.pane.Plotly(self.atdFig)
        self.peakDFPane = pn.pane.DataFrame(self.peaksDF)
    
    def updateMainPlot(self, updatePeakHandler = True):
        self.atdData.x = self.x
        self.atdData.y = self.y
        if updatePeakHandler:
            self.peakHandler.clearData()
            self.peakHandler.setData(self.atdData.y)

    
    def setPlotContainer(self):
        self.plotBox = pn.Column(
                                self.mzPane,
                                self.extractBox,
                                self.timePane,
                                self.processingBox,
                                self.atdPane,
                                self.findPeakBox,
                                self.peakDFPane,
                                self.debugArea
                                )
    def getPlotContainer(self):
        return self.plotBox      


    
    def loadButtonClick(self, event = None):
        fileName = self.fileSelector.value
        
        print(fileName)
        if len(fileName)>0:
            fileName = fileName[0]
            self.debugArea.value = str(fileName)
        else:
            fileName = None
        #add check for file status (e.g. os.path.isfile)
        if fileName != None:
            if os.path.isfile(fileName):
                
                if self.orbiBool.value:
#                     self.x, self.y, self.spectral_data = loadOrbiMS(fileName)
                    self.x, self.y, self.spectral_data = loadOrbiMS_v2(fileName, 0.1)#need to add this as a parameter
                else:
                    self.x, self.y, self.spectral_data = loadMS(fileName)
                    
    #                 self.x = np.arange(len(self.y))

                self.mzTrace.x = self.x
                self.mzTrace.y = self.y
                
                self.peakTrace.x = []
                self.peakTrace.y = []
                
#                 self.fig.data = self.fig.data[:2]#clear previous fits
                
                self.mzFig.update_layout(title=os.path.basename(fileName))
#                 self.dfWidget.clear_output()#Clear any previous peak fit results
                self.loadBool = True
                self.fileName = fileName
                self.peakHandler.clearData()
                self.resetATDButtonClick()
        else:
            print("No file was Selected!!!")
            self.loadBool = False

    def extractButtonClick(self, event=None):
#         mz1, mz2 = self.mzFig.layout.xaxis.range
        mz1, mz2 = self.mzPane.viewport['xaxis.range']
        mzCenter = (mz1+mz2)/2
        mzTol = mz2 - mzCenter
        
        
        if self.orbiBool.value:
            self.intfX, self.intfY = getXIC_Orbi(self.spectral_data, mzCenter, tol = mzTol, secondsBool = True)
        else:
            self.intfX, self.intfY = getXIC(self.spectral_data, mzCenter, tol=mzTol)
        
        
        self.timeData.x = self.intfX
        self.timeData.y = self.intfY
        
        self.timeFig.update_layout(title='*RAW* XIC from selected mass range: {:.2f}-{:.2f} m/z'.format(mz1, mz2))            

    def updateAxisButtonClick(self, event=None):
        l = self.lowerXInput.value
        u = self.upperXInput.value
        
        self.mzFig.update_xaxes(range=[l,u])        

        
    def applyFTButtonClick(self, event=None):
        
#         self.dfWidget.clear_output() #clear previous df output
        
#         #clearing any peak markers in case peakfind has been used before ATD converstion
#         self.peakTrace.x = []
#         self.peakTrace.y = []
#         self.atdFig.data = self.fig3.data[:2] #this clears the fits

        self.modIntfY = Mod_Intf(time = self.intfX,
                                 amp = self.intfY,
                                 window = self.windowTypeSelector.value,
                                 padLen = self.padWidget.value,
                                 minBool = False,
                                 windowBool = self.windowCB.value,
                                 padBool = self.zeroPadCB.value)
        
        
        self.timeData.y = self.modIntfY
        self.timeData.x = np.arange(len(self.timeData.y))

        self.atdX, self.atdY = DFT_data(time = self.intfX,
                                        amp = self.intfY,
                                        window = self.windowTypeSelector.value,
                                        padLen = self.padWidget.value,
                                        minBool = False,
                                        windowBool = self.windowCB.value,
                                        padBool = self.zeroPadCB.value)
        
        #ignoring the first giant peak of the mobility spectrum (not important)
        #doing so allows the mobility spectrum to have reasonable y-axis limits
        self.atdData.x = self.atdX[self.pointCutoffInput.value:]#FIXME
        self.atdData.y = self.atdY[self.pointCutoffInput.value:]#FIXME
        
        self.atdFig.update_xaxes(title='Frequency (Hz)')
        self.atdFig.update_yaxes(title='Intensity (a.u.)')
        
        
        self.atdFig.update_layout(title='Frequency domain plot')
        
        mz1, mz2 = self.mzPane.viewport['xaxis.range']
        self.timeFig.update_layout(title='*Interpolated* XIC: {:.2f}-{:.2f} m/z'.format(mz1, mz2))
        self.timeFig.update_xaxes(title='Bins')        

    
    def ATDButtonClick(self, event = None):

        #clearing any peak markers in case peakfind has been used before ATD converstion
        self.peakTrace.x = []
        self.peakTrace.y = []
        self.atdFig.data = self.atdFig.data[:1]#2] #this clears the fits

        
        sweepRate = self.sweepRateWidg.value
        
        atd = convertHztoATD(self.atdX, sweepRate)
        
        self.atdData.x = atd[self.pointCutoffInput.value:] #FIXME
        self.atdData.y = self.atdY[self.pointCutoffInput.value:] #FIXME
        
        self.atdFig.update_xaxes(title='Time (ms)')
        
        self.atdFig.update_layout(title='Time domain plot')
        
        dfDict = {}
        dfDict['Peaks Not Set'] = []
        emptyDF = pd.DataFrame.from_dict(dfDict)
        self.peakHandler.setResultDF(emptyDF)
        
        self.updatePeaksDF()               
        
        
    def resetATDButtonClick(self, event=None):
         
#         self.dfWidget.clear_output()

        #clearing any peak markers in case peakfind has been used before ATD converstion
        self.peakTrace.x = []
        self.peakTrace.y = []
        self.atdFig.data = self.atdFig.data[:1]#2] #this clears the fits
        
        #reset the plot back to the frequency domain plot given by FT
        self.atdData.x = self.atdX[DCCUTOFF:] #FIXME
        self.atdData.y = self.atdY[DCCUTOFF:] #FIXME
        self.atdFig.update_xaxes(title='Frequency (Hz)')
        
        self.atdFig.update_layout(title='Frequency domain plot')
        
        dfDict = {}
        dfDict['Peaks Not Set'] = []
        emptyDF = pd.DataFrame.from_dict(dfDict)
        self.peakHandler.setResultDF(emptyDF)
        
        self.updatePeaksDF()        
                                                      
                                                      
                                                      

    # Find peaks functions##############################################
    ####################################################################
    def findPButtonClick(self, event = None):
#         self.dfWidget.clear_output()
        
        if self.loadBool:
            
            self.atdFig.data = self.atdFig.data[:2]#clear previous fits

            self.peakHandler.setData(self.atdData.y)
            
            curD = self.distanceSlider.value
            curW = self.widthSlider.value
            curH = self.minHSlider.value
            
            self.peakHandler.getPeakLocs(curD, curW, curH)
            
            self.updatePeakLoc()
            
    def updatePeakLoc(self, event = None):
        if self.loadBool:
            if self.peakHandler.peakBool:
                pInd = self.peakHandler.peakInds
                
                self.atdFig.add_trace(go.Scatter(
                    x=[],
                    y=np.array([]),
                        name="Peak Location",
                        mode="markers",
                        marker = dict(
                            color = 'rgb(255,255,255)',
                            size = 8,
                            line = dict(
                              color = 'rgb(255,0,0)',
                              width = 2
                            )
                          )
                        )
                )

                self.peakTrace = self.atdFig.data[1]                 
                
                self.peakTrace.x = self.atdData.x[pInd]
                self.peakTrace.y = self.atdData.y[pInd]        

    def updatePeaksDF(self):
#         newDF = pd.DataFrame.from_dict(dp.peakHandler.fitResults)
#         newDF = newDF[newDF.columns.difference(['XARRAYS'])]#Exclude the raw arrays from the dataframe
#         newDF = newDF[newDF.columns.difference(['YARRAYS'])]#Exclude the raw arrays from the dataframe        
#         self.peaksDF = newDF
        self.peaksDF = self.peakHandler.resultDF
        self.peakDFPane.object = self.peaksDF                
                
    def fitPeaks(self):
        
        mparams = []
        paramKey = ['Length (cm)', 'Voltage (V)', 'Temperature (K)', 'Pressure (torr)']
        for param in paramKey:
            mparams.append(self.controlDict[param].value)
        
        res = fitPeaksWorkflow(self.atdData.x, 
                               self.atdData.y, 
                               self.peakHandler.peakInds, 
                               self.peakHandler.peakProperties,
                               fitFactor = self.peakWinSlider.value,#determines the width around the peak to fit
                               debug = False)
        
        # return curDF, fitResults, params, scaleFactor
        curDF, fitResults, params, scaleFactor = res #unpack results
        
        self.peakHandler.setResults(fitResults)
        self.peakHandler.setResultDF(curDF)
        self.updatePeaksDF()
        self.plotFits()

    def plotFits(self, updateDFBool = True):
        '''
        setting self.fig.data = [] clears the plot
        
        '''
#         if len(self.fig.data)>2:
            
        self.atdFig.data = self.atdFig.data[:2]#clear previous fits    
        for i,c in enumerate(self.peakHandler.fitResults['CENTROIDS']):
            if c>0:
                x = self.peakHandler.fitResults['XARRAYS'][i]
                y = self.peakHandler.fitResults['YARRAYS'][i]
                self.atdFig.add_trace(go.Scatter(
                    x=x,
                    y=np.array(y),
                        name="Peak Fit",
                        mode="lines",
                        )
                )
#         if updateDFBool:
#             self.updateDFOutput()        
        
                
    def fitPButtonClick(self, event = None):
        self.exportBool = True
        if self.loadBool:
            if self.peakHandler.peakBool:
                self.fitPeaks()        

In [63]:
vanilla = pn.template.VanillaTemplate(title='FT-IM Processor')

pn.config.sizing_mode = 'stretch_width'

# atdAccord = ATDBOX()
# peakAccord = PEAKBOX()

dp = DataPlotter()
dp.setPanes()
dp.setPlotContainer()

vanilla.sidebar.append(dp.atdAccord)
vanilla.sidebar.append(dp.peakAccord)
vanilla.sidebar.append(dp.dataAccord)
# vanilla.sidebar.append(FILEACCORD(dp.fileBox))


tabs = pn.Tabs(('Data Viewer', dp.getPlotContainer()),('File Selection', dp.fileBox))
tabs

vanilla.main.append(
    tabs
)

# vanilla
vanilla.show()
# vanilla.servable();

Launching server at http://localhost:57962


['C:\\Users\\bhclo\\Desktop\\GoogleDrive\\My Drive\\SPELLS\\Panel\\LeuEnkTXA_StpFrq_7505Hz_500scan_210702100806.mzML']
popt: [2.09656455 0.98067944 0.01347696]
Peak 1 @ 2096.565 ms, Rp = 66.06
popt: [2.09656455 0.98067939 0.01347696 2.55888616 0.2025134  0.0140452 ]
Peak 1 @ 2096.565 ms, Rp = 66.06
Peak 2 @ 2558.886 ms, Rp = 77.37
popt: [2.09656455 0.98067939 0.01347696 2.55888616 0.2025134  0.0140452 ]
Peak 1 @ 2096.565 ms, Rp = 66.06
Peak 2 @ 2558.886 ms, Rp = 77.37


In [155]:
print(dp.fileName)

None


In [101]:
#dp.fitPeaks()

In [102]:
pn.__version__

'0.11.3'